In [1]:
import os
import glob
import arviz as az
import numpy as np
import pandas as pd
from pytensor.tensor import TensorVariable
from typing import Optional, Tuple
import pytensor.tensor as pt

import pymc as pm
import jax
#import seaborn as sns
# import aesara
# import aesara.tensor as at
# import pytensor.tensor as pt
import matplotlib.pyplot as plt
import xarray as xr
import re
from scipy import stats
from scipy.special import logit
from scipy.special import expit as logist
from matplotlib.axes import Axes
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
from typing import List
from pandas import DataFrame
import sys
import pickle

RANDOM_SEED = 8927
rng = np.random.default_rng(RANDOM_SEED)

jax.default_backend()
jax.devices()

[cuda(id=0)]

In [2]:
path_out = './pymc_model'
with open(os.path.join('pymc_model','brain_16mm_vshd.pkl'),'rb') as f:
    inputBrain16um = pickle.load(f)
df = inputBrain16um['dsg']
inf_aver = inputBrain16um['wsf']

In [4]:
g_gf = inf_aver.to_numpy()
d_sg = df.to_numpy()

In [5]:
g_gf = g_gf[:3000,:]
d_sg = d_sg[:10_000,:3000]
d_sg.shape,g_gf.shape

((10000, 3000), (3000, 13))

In [6]:
# ngens = len(inf_aver)
# nspost = len(df)
# nfeat = len(inf_aver.columns)

ngens = 3000
nspost = 10_000
nfeat = 13

In [7]:
g_gf.shape, d_sg.shape

((3000, 13), (10000, 3000))

In [9]:
# g_gf = pm.Minibatch(g_gf,batch_size = 100)
# d_sg = pm.Minibatch(d_sg,batch_size = 100)
# All variables shape[0] in Minibatch should be equal, check your Minibatch(data1, data2, ...) code
#d_sg = d_sg.T
g_gf_, d_sg_ = pm.Minibatch(g_gf, d_sg, batch_size=10)


In [19]:
with pm.Model() as simNB_model:
    mu_m = pm.Gamma("mu_m",1,1)
    o_m = pm.Exponential("o_m",3)
    a_m = pm.Deterministic("a_m",1/o_m**2)
    m_g = pm.Gamma("m_g",a_m,a_m/mu_m,shape = (1,ngens))
    
    a_y = pm.Gamma("a_y",10,10/20)
    y_s = pm.Gamma("y_s",10,10/30,shape = (nspost,1))
    
    b0 = pm.Gamma("b0",9,3,shape = 2)
    o_g = pm.Exponential("o_g",b0[0], shape = (nspost,1))
    a_g = pm.Deterministic("a_g",1/o_g**2)
    
    o_es = pm.Exponential("o_es",b0[1])
    a_es = pm.Deterministic("a_es",1/o_es**2)
    mu_es = pm.Gamma("mu_es",1,100)
    s_eg = pm.Gamma("s_eg",a_es,a_es/mu_es,shape = (1,ngens))
    
    
    k_r  = pm.Gamma("k_r",7,1)
    x_rf  = pm.Gamma("x_rf",k_r/50,k_r,shape = (50,nfeat))
    B_s = pm.Gamma("B_s",7,1) 
    N_s =pm.Gamma("N_s",4*5,5) #4 cells per spot vn =5 acording to seggestion of section 2.1  
    z_sr  = pm.Gamma("z_sr",B_s/50,1/(N_s/B_s),shape = (nspost,50))
    mu_sf = pm.Deterministic("mu_sf",pm.math.dot(z_sr,x_rf))
    w_sf = pm.Gamma("w_sf",alpha = mu_sf.eval()*10, beta =10)
    
    musg1_p1 = pm.math.dot(w_sf,g_gf.T)
    musg1_p2 = m_g * musg1_p1
    musg1_p3 = musg1_p2+s_eg
    musg1_p4 = pm.Deterministic("musg1_p4",musg1_p3*y_s)
    
    count = pm.NegativeBinomial("count", mu=musg1_p4, alpha=a_g, observed=d_sg)
    #trace = pm.sample(draws=100,tune = 10, target_accept=0.95, random_seed=RANDOM_SEED, chains = 6)

In [20]:
with simNB_model:
    approx = pm.fit(100000, callbacks=[pm.callbacks.CheckParametersConvergence(tolerance=1e-4)])
    idata_advi = az.from_pymc3(approx.sample(500))
    

/opt/conda/lib/python3.11/site-packages/pytensor/tensor/rewriting/shape.py:169: UserWarning: Failed to infer_shape from Op Elemwise{mul,no_inplace}.
Input shapes: [(TensorConstant{1}, TensorConstant{3000}), (TensorConstant{10000}, TensorConstant{100})]
Exception encountered during infer_shape: <class 'ValueError'>
Exception message: Could not broadcast dimensions
Traceback: Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/pytensor/tensor/rewriting/shape.py", line 145, in get_node_infer_shape
    o_shapes = shape_infer(
               ^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/pytensor/tensor/elemwise.py", line 834, in infer_shape
    out_shape = pytensor.tensor.broadcast_shape(*i_shapes, arrays_are_shapes=True)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/pytensor/tensor/extra_ops.py", line 1455, in broadcast_shape
    return broadcast_shape_iter(arr

ValueError: Input dimension mismatch. One other input has shape[1] = 3000, but input[1].shape[1] = 100.
Apply node that caused the error: Elemwise{Composite{((i0 * i1) + i2)}}(m_g_log___log, dot.0, s_eg_log___log)
Toposort index: 149
Inputs types: [TensorType(float64, (1, 3000)), TensorType(float64, (10000, ?)), TensorType(float64, (1, 3000))]
Inputs shapes: [(1, 3000), (10000, 100), (1, 3000)]
Inputs strides: [(24000, 8), (800, 8), (24000, 8)]
Inputs values: ['not shown', 'not shown', 'not shown']
Outputs clients: [[Elemwise{mul,no_inplace}(Elemwise{Composite{((i0 * i1) + i2)}}.0, y_s_log___log), Elemwise{Composite{(i0 + (i1 * i2 * i3))}}(TensorConstant{(1, 1) of 1.0}, Elemwise{Composite{((i0 * i1) + i2)}}.0, y_s_log___log, Elemwise{sqr,no_inplace}.0), Elemwise{mul,no_inplace}(Elemwise{Composite{((i0 * i1) + i2)}}.0, y_s_log___log, Elemwise{add,no_inplace}.0), Elemwise{Composite{((Switch(i0, i1, Switch(i2, i1, Switch(i3, i1, (i4 / i5)))) + Switch(i6, i1, Switch(i2, i1, (i7 / i5))) + S

/opt/conda/lib/python3.11/site-packages/pytensor/tensor/rewriting/shape.py:169: UserWarning: Failed to infer_shape from Op Elemwise{mul,no_inplace}.
Input shapes: [(TensorConstant{10000}, TensorConstant{3000}), (TensorConstant{100}, TensorConstant{3000})]
Exception encountered during infer_shape: <class 'ValueError'>
Exception message: Could not broadcast dimensions
Traceback: Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/pytensor/tensor/rewriting/shape.py", line 145, in get_node_infer_shape
    o_shapes = shape_infer(
               ^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/pytensor/tensor/elemwise.py", line 834, in infer_shape
    out_shape = pytensor.tensor.broadcast_shape(*i_shapes, arrays_are_shapes=True)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/pytensor/tensor/extra_ops.py", line 1455, in broadcast_shape
    return broadcast_shape_iter(

/opt/conda/lib/python3.11/site-packages/pytensor/tensor/rewriting/shape.py:169: UserWarning: Failed to infer_shape from Op Elemwise{add,no_inplace}.
Input shapes: [(TensorConstant{100}, TensorConstant{3000}), (TensorConstant{10000}, TensorConstant{1})]
Exception encountered during infer_shape: <class 'ValueError'>
Exception message: Could not broadcast dimensions
Traceback: Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/pytensor/tensor/rewriting/shape.py", line 145, in get_node_infer_shape
    o_shapes = shape_infer(
               ^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/pytensor/tensor/elemwise.py", line 834, in infer_shape
    out_shape = pytensor.tensor.broadcast_shape(*i_shapes, arrays_are_shapes=True)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/pytensor/tensor/extra_ops.py", line 1455, in broadcast_shape
    return broadcast_shape_iter(arr

/opt/conda/lib/python3.11/site-packages/pytensor/tensor/rewriting/shape.py:169: UserWarning: Failed to infer_shape from Op Elemwise{switch,no_inplace}.
Input shapes: [(TensorConstant{100}, TensorConstant{3000}), (TensorConstant{1}, TensorConstant{1}), (TensorConstant{10000}, TensorConstant{3000})]
Exception encountered during infer_shape: <class 'ValueError'>
Exception message: Could not broadcast dimensions
Traceback: Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/pytensor/tensor/rewriting/shape.py", line 145, in get_node_infer_shape
    o_shapes = shape_infer(
               ^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/pytensor/tensor/elemwise.py", line 834, in infer_shape
    out_shape = pytensor.tensor.broadcast_shape(*i_shapes, arrays_are_shapes=True)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/pytensor/tensor/extra_ops.py", line 1455, in broad

...

/opt/conda/lib/python3.11/site-packages/pytensor/tensor/rewriting/shape.py:169: UserWarning: Failed to infer_shape from Op Elemwise{switch,no_inplace}.
Input shapes: [(TensorConstant{100}, TensorConstant{3000}), (TensorConstant{1}, TensorConstant{1}), (TensorConstant{10000}, TensorConstant{3000})]
Exception encountered during infer_shape: <class 'ValueError'>
Exception message: Could not broadcast dimensions
Traceback: Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/pytensor/tensor/rewriting/shape.py", line 145, in get_node_infer_shape
    o_shapes = shape_infer(
               ^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/pytensor/tensor/elemwise.py", line 834, in infer_shape
    out_shape = pytensor.tensor.broadcast_shape(*i_shapes, arrays_are_shapes=True)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/pytensor/tensor/extra_ops.py", line 1455, in broad

/opt/conda/lib/python3.11/site-packages/pytensor/tensor/rewriting/shape.py:169: UserWarning: Failed to infer_shape from Op Elemwise{switch,no_inplace}.
Input shapes: [(TensorConstant{100}, TensorConstant{3000}), (TensorConstant{1}, TensorConstant{1}), (TensorConstant{10000}, TensorConstant{3000})]
Exception encountered during infer_shape: <class 'ValueError'>
Exception message: Could not broadcast dimensions
Traceback: Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/pytensor/tensor/rewriting/shape.py", line 145, in get_node_infer_shape
    o_shapes = shape_infer(
               ^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/pytensor/tensor/elemwise.py", line 834, in infer_shape
    out_shape = pytensor.tensor.broadcast_shape(*i_shapes, arrays_are_shapes=True)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/pytensor/tensor/extra_ops.py", line 1455, in broad

/opt/conda/lib/python3.11/site-packages/pytensor/tensor/rewriting/shape.py:169: UserWarning: Failed to infer_shape from Op Elemwise{Composite{Switch(i0, Switch(i1, Switch(i2, i3, Switch(i4, i5, (Switch(i6, i7, (i8 * log(i9))) - (i10 + i9)))), i5), Switch(i11, Switch(i4, i5, (((gammaln(i12) - i10) - i13) + Switch(i14, i7, (i8 * log(i15))) + Switch(i16, i17, (i18 / i19)))), i5))}}.
Input shapes: [(TensorConstant{10000}, TensorConstant{1}), (TensorConstant{1}, TensorConstant{1}), (TensorConstant{10000}, TensorConstant{3000}), (TensorConstant{1}, TensorConstant{1}), (TensorConstant{100}, TensorConstant{3000}), (TensorConstant{1}, TensorConstant{1}), (TensorConstant{10000}, TensorConstant{3000}), (TensorConstant{100}, TensorConstant{3000}), (TensorConstant{100}, TensorConstant{3000}), (TensorConstant{10000}, TensorConstant{3000}), (TensorConstant{100}, TensorConstant{3000}), (TensorConstant{1}, TensorConstant{1}), (TensorConstant{10000}, Shape_i{1}.0), (TensorConstant{10000}, TensorConstant

/opt/conda/lib/python3.11/site-packages/pytensor/tensor/rewriting/shape.py:169: UserWarning: Failed to infer_shape from Op Elemwise{Composite{((Switch(i0, i1, Switch(i2, i1, Switch(i3, i1, (i4 / i5)))) + Switch(i6, i1, Switch(i2, i1, (i7 / i5))) + Switch(i6, i1, Switch(i2, i1, Switch(i8, i1, ((i9 * i10 * i9) / i11)))) + Switch(i12, i1, Switch(i2, i1, Switch(i8, i1, (((i13 + (i9 / i14)) / i14) / i11))))) - Switch(i2, i1, Switch(i3, i1, Switch(i8, (-i9), i1))))}}[(0, 4)].
Input shapes: [(TensorConstant{10000}, TensorConstant{3000}), (TensorConstant{1}, TensorConstant{1}), (TensorConstant{100}, TensorConstant{3000}), (TensorConstant{10000}, TensorConstant{3000}), (TensorConstant{10000}, TensorConstant{3000}), (TensorConstant{10000}, TensorConstant{3000}), (TensorConstant{10000}, TensorConstant{3000}), (TensorConstant{10000}, TensorConstant{3000}), (TensorConstant{10000}, TensorConstant{1}), (TensorConstant{10000}, TensorConstant{3000}), (TensorConstant{100}, TensorConstant{3000}), (Tensor

ValueError: Input dimension mismatch. One other input has shape[1] = 3000, but input[1].shape[1] = 100.
Apply node that caused the error: Elemwise{Composite{((i0 * i1) + i2)}}(m_g_log___log, dot.0, s_eg_log___log)
Toposort index: 149
Inputs types: [TensorType(float64, (1, 3000)), TensorType(float64, (10000, ?)), TensorType(float64, (1, 3000))]
Inputs shapes: [(1, 3000), (10000, 100), (1, 3000)]
Inputs strides: [(24000, 8), (800, 8), (24000, 8)]
Inputs values: ['not shown', 'not shown', 'not shown']
Outputs clients: [[Elemwise{mul,no_inplace}(Elemwise{Composite{((i0 * i1) + i2)}}.0, y_s_log___log), Elemwise{Composite{(i0 + (i1 * i2 * i3))}}(TensorConstant{(1, 1) of 1.0}, Elemwise{Composite{((i0 * i1) + i2)}}.0, y_s_log___log, Elemwise{sqr,no_inplace}.0), Elemwise{mul,no_inplace}(Elemwise{Composite{((i0 * i1) + i2)}}.0, y_s_log___log, Elemwise{add,no_inplace}.0), Elemwise{Composite{((Switch(i0, i1, Switch(i2, i1, Switch(i3, i1, (i4 / i5)))) + Switch(i6, i1, Switch(i2, i1, (i7 / i5))) + S